In [1]:
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

import pandas as pd
pd.options.display.max_columns = 100

from matplotlib import pyplot as plt
import numpy as np

import seaborn as sns
sns.set(rc={'figure.figsize':(12,9)})
import pylab as plot
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss,accuracy_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
from xgboost import XGBClassifier
import xgboost as xgb

In [3]:
#for scaling
from sklearn.preprocessing import StandardScaler

Importing training dataset

In [4]:
data = pd.read_csv('./train.csv')

In [5]:
print(data.shape)

(867873, 10)


In [6]:
data.columns

Index(['Dates', 'Category', 'Descript', 'DayOfWeek', 'PdDistrict',
       'Resolution', 'Address', 'X', 'Y', 'Id'],
      dtype='object')

Splitting the "Dates" into date and time

In [7]:
df=data
datetime=df.Dates.str.split(pat=" ",expand=True)
datetime.columns=['Date','Time']
#datetime

In [8]:
Date=datetime.Date.str.split(pat="-",expand=True)
Date.columns=['Year','Month','Day']

Time=datetime.Time.str.split(pat=":",expand=True)
Time.columns=['Hour','Minute','Second']

Dropping the column of "Dates" and appending the dataframe "datetime"

In [9]:
df=pd.concat([df,Date,Time],axis=1)
#df

In [10]:
df=df.drop(labels=['Dates'],axis=1)

# #Encoding

In [11]:
df.columns

Index(['Category', 'Descript', 'DayOfWeek', 'PdDistrict', 'Resolution',
       'Address', 'X', 'Y', 'Id', 'Year', 'Month', 'Day', 'Hour', 'Minute',
       'Second'],
      dtype='object')

LabelEncoding of "DayOfWeek" and "PdDistrict"

In [12]:
le = preprocessing.LabelEncoder()

In [13]:
#y=pd.get_dummies(df.Category,columns=['Category'],prefix=" ",prefix_sep=" ",drop_first=True,)

le_res=le.fit_transform(df['Category'])
y=pd.DataFrame(le_res)
y.columns=['Category']
#y

# Hours clubbing

In [14]:
"""df['Hour']=df['Hour'].apply(lambda x:int(x))
#df['Hour']=df['Hour'].apply(lambda x : 'EARLY_Morning' if (x >= 1) and (x <=5) else ('MORNING' if x >= 6 and x <=11 else ('DAY' if x>=12 and x<=18 else 'Night')))
df['Hour']=df['Hour'].apply(lambda x : 'EARLY_MORNING' if x in (1,2,3,4,5) else ('MORNING' if x in (6,7,8,9,10,11) else ('DAY' if x in (12,13,14,15,16,17,18) else 'NIGHT')))"""

"df['Hour']=df['Hour'].apply(lambda x:int(x))\n#df['Hour']=df['Hour'].apply(lambda x : 'EARLY_Morning' if (x >= 1) and (x <=5) else ('MORNING' if x >= 6 and x <=11 else ('DAY' if x>=12 and x<=18 else 'Night')))\ndf['Hour']=df['Hour'].apply(lambda x : 'EARLY_MORNING' if x in (1,2,3,4,5) else ('MORNING' if x in (6,7,8,9,10,11) else ('DAY' if x in (12,13,14,15,16,17,18) else 'NIGHT')))"

# Month Clubbing

In [15]:
"""df['Month']=df['Month'].apply(lambda x:int(x))
#df['Month']=df['Month'].apply(lambda x : 'MonthLow' if x== 12 else ('MonthMed' if x in (2,6,7,8,9,11) else 'MonthHigh'))
df['Month']=df['Month'].apply(lambda x : 'Winter' if x in (11,12,1,2) else ('Spring' if x in (3,4,5) else('Summer' if x in (6,7,8) else('Fall'))))"""

"df['Month']=df['Month'].apply(lambda x:int(x))\n#df['Month']=df['Month'].apply(lambda x : 'MonthLow' if x== 12 else ('MonthMed' if x in (2,6,7,8,9,11) else 'MonthHigh'))\ndf['Month']=df['Month'].apply(lambda x : 'Winter' if x in (11,12,1,2) else ('Spring' if x in (3,4,5) else('Summer' if x in (6,7,8) else('Fall'))))"

In [16]:
df['Month'].nunique()

12

# Minute Clubbing 

In [17]:
"""df['Minute']=df['Minute'].apply(lambda x:int(x))
df['Minute']=df['Minute'].apply(lambda x : 'low' if x <31 else 'high')"""

"df['Minute']=df['Minute'].apply(lambda x:int(x))\ndf['Minute']=df['Minute'].apply(lambda x : 'low' if x <31 else 'high')"

In [18]:
df.columns

Index(['Category', 'Descript', 'DayOfWeek', 'PdDistrict', 'Resolution',
       'Address', 'X', 'Y', 'Id', 'Year', 'Month', 'Day', 'Hour', 'Minute',
       'Second'],
      dtype='object')

# DayOfWeek

In [19]:
"""df['DayOfWeek']= df['DayOfWeek'].apply(lambda x : 'WeekHigh' if x in (3,5) else ('WeekMed' if x in (2,4,6) else 'WeekLow'))
"""

"df['DayOfWeek']= df['DayOfWeek'].apply(lambda x : 'WeekHigh' if x in (3,5) else ('WeekMed' if x in (2,4,6) else 'WeekLow'))\n"

# Address Update

In [20]:
df['Intersection']=df['Address'].apply(lambda x : 1 if '/' in x else 0)
df['Block']=df['Address'].apply(lambda x : 1 if 'Block' in x else 0)
#df['StreetSuffix']=df['Address'].apply(lambda x : x.split(' ')[-1] if len(x.split(' ')[-1])==2 else 0) 

In [21]:
df.head()

,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Id,Year,Month,Day,Hour,Minute,Second,Intersection,Block
0,SEX OFFENSES FORCIBLE,"FORCIBLE RAPE, BODILY FORCE",Friday,MISSION,NONE,2100 Block of MISSION ST,-122.419331,37.762264,141546,2013,06,28,17,40,00,0,1
1,LIQUOR LAWS,CONSUMING ALCOHOL IN PUBLIC VIEW,Thursday,SOUTHERN,"ARREST, BOOKED",1000 Block of MARKET ST,-122.411340,37.781271,794152,2004,02,19,02,46,00,0,1
2,FRAUD,"CREDIT CARD, THEFT BY USE OF",Wednesday,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403405,37.775421,531205,2007,11,14,00,01,00,0,1
3,ROBBERY,ROBBERY OF A CHAIN STORE WITH A GUN,Thursday,BAYVIEW,DISTRICT ATTORNEY REFUSES TO PROSECUTE,2400 Block of SAN BRUNO AV,-122.404715,37.730161,523137,2007,12,27,18,30,00,0,1
4,OTHER OFFENSES,PROBATION VIOLATION,Sunday,SOUTHERN,"ARREST, BOOKED",4TH ST / STEVENSON ST,-122.405239,37.785265,200968,2012,09,09,17,02,00,1,0


# Address splitting

In [22]:
address=pd.DataFrame(df['Address'],columns=['Address'])
address=address.Address.str.split(pat=" /",expand=True )

address.columns=['Address','Intr2']

address=address.Address.str.split(pat=" /",expand=True )
address.columns=['Address']

In [23]:
address.head()

,Address
0,2100 Block of MISSION ST
1,1000 Block of MARKET ST
2,800 Block of BRYANT ST
3,2400 Block of SAN BRUNO AV
4,4TH ST


In [24]:
string=address.iloc[:,0]
string=string.str.strip()

In [25]:
address_fram=string.to_frame()
temp=address_fram['Address'].astype(str).str[-2:]

In [26]:
temp.unique()

array(['ST', 'AV', 'LN', 'DR', 'BL', 'HY', 'CT', 'RD', 'PL', 'PZ', '80',
       'TR', 'WY', 'AL', 'AY', 'ER', 'MS', 'CR', 'TI', 'WK', 'AR', 'EX',
       'NO', 'RK', 'RW'], dtype=object)

In [27]:
address=temp.to_frame()

("ST","AV","LN","DR","BL","HY","CT","RD","PL","PZ","TR","AL","CR","WK","EX","RW")

In [28]:
address['Address']=address['Address'].apply(lambda x :( x if x in ("ST","AV","LN","DR","BL","HY","CT","RD","PL","PZ","TR","AL","CR","WK","EX","RW") else (("I-80" if x in ("80") else ("HWY" if x in ("WY") else ("WAY" if x in ("AY") else ("TER" if x in ("ER") else ("ALMS" if x in ("MS") else ("MAR" if x in ("AR") else ("PARK" if x in ("RK") else ("STWY" if x in ("WY") else ("VIA" if x in ("NO") else ("BLOCK")))))))))))))

In [29]:
#address.to_csv("Address.csv", float_format = '%.5F')

In [30]:
df=df.drop(labels=['Address'],axis=1)

Concatenating "df" and "address"

In [31]:
df=pd.concat([address,df],axis=1)

In [32]:
Id=df['Id']
df=df.drop(['Descript','Category','Resolution','Id','Second',],axis=1)

In [33]:
df.head()

,Address,DayOfWeek,PdDistrict,X,Y,Year,Month,Day,Hour,Minute,Intersection,Block
0,ST,Friday,MISSION,-122.419331,37.762264,2013,06,28,17,40,0,1
1,ST,Thursday,SOUTHERN,-122.411340,37.781271,2004,02,19,02,46,0,1
2,ST,Wednesday,SOUTHERN,-122.403405,37.775421,2007,11,14,00,01,0,1
3,AV,Thursday,BAYVIEW,-122.404715,37.730161,2007,12,27,18,30,0,1
4,ST,Sunday,SOUTHERN,-122.405239,37.785265,2012,09,09,17,02,1,0


Label Encoded Columns :PdDistrict ,Address ,X ,Y

In [34]:
data['PdDistrict'].unique()

array(['MISSION', 'SOUTHERN', 'BAYVIEW', 'CENTRAL', 'INGLESIDE',
       'NORTHERN', 'RICHMOND', 'TARAVAL', 'TENDERLOIN', 'PARK'],
      dtype=object)

# Label Encoding

In [35]:
le = preprocessing.LabelEncoder()

"DayOf Week"

In [36]:
le_res=le.fit_transform(df['DayOfWeek'])
Day=pd.DataFrame(le_res)
Day.columns=['DayOfWeek']
df=df.drop(labels=['DayOfWeek'],axis=1)
df=pd.concat([Day,df],axis=1)


"PdDistrict"

In [37]:

#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['PdDistrict'])
District=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
District.columns=['District']
df=df.drop(labels=['PdDistrict'],axis=1)
df=pd.concat([District,df],axis=1)



"Address"

In [38]:

le_res=le.fit_transform(df['Address'])
Address=pd.DataFrame(le_res)
Address.columns=['Address']
df=df.drop(labels=['Address'],axis=1)
df=pd.concat([Address,df],axis=1)



"Year"

In [39]:

#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Year'])
Year=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Year.columns=['Year']
df=df.drop(labels=['Year'],axis=1)
df=pd.concat([Year,df],axis=1)



"Month"

In [40]:

#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Month'])
Month=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Month.columns=['Month']
df=df.drop(labels=['Month'],axis=1)
df=pd.concat([Month,df],axis=1)



"Day"

In [41]:

#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Day'])
Day=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Day.columns=['Day']
df=df.drop(labels=['Day'],axis=1)
df=pd.concat([Day,df],axis=1)



"Hour"

In [42]:

#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Hour'])
Hour=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Hour.columns=['Hour']
df=df.drop(labels=['Hour'],axis=1)
df=pd.concat([Hour,df],axis=1)



"Minute"

In [43]:

#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Minute'])
Minute=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Minute.columns=['Minute']
df=df.drop(labels=['Minute'],axis=1)
df=pd.concat([Minute,df],axis=1)


# One Hot Encoding

In [44]:
"""df=pd.get_dummies(df,columns=['DayOfWeek','PdDistrict','Year','Month','Day','Hour','Minute','Address'],drop_first=True)"""

"df=pd.get_dummies(df,columns=['DayOfWeek','PdDistrict','Year','Month','Day','Hour','Minute','Address'],drop_first=True)"

In [45]:
df

,Minute,Hour,Day,Month,Year,Address,District,DayOfWeek,X,Y,Intersection,Block
0,40,17,27,5,10,19,3,0,-122.419331,37.762264,0,1
1,46,2,18,1,1,19,7,4,-122.411340,37.781271,0,1
2,1,0,13,10,4,19,7,6,-122.403405,37.775421,0,1
3,30,18,26,11,4,2,0,4,-122.404715,37.730161,0,1
4,2,17,8,8,9,19,7,3,-122.405239,37.785265,1,0
5,14,6,13,4,11,19,3,6,-122.418405,37.766760,1,0
6,45,23,8,7,3,12,1,6,-122.404651,37.790442,0,1
7,41,12,15,10,9,19,2,0,-122.410871,37.737526,0,1
8,53,23,4,1,4,19,0,1,-122.405911,37.732964,1,0
9,41,7,6,1,3,19,4,5,-122.417710,37.784236,1,0


# Feature Scaling(Not used YET)

In [46]:
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2,f_classif,SelectPercentile,SelectFpr
test = SelectFpr(chi2, alpha=0.01).fit_transform(df, y)
# summarize scores

print(test[0:50,:])

ValueError: Input X must be non-negative.

36 Crime Category are there 

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
#weight={Address:3,District:3,X:1,Day:2}
#weight={LARCENY/THEFT:35}
#estimator = LogisticRegression(penalty='l2',random_state = 0,class_weight='balanced',multi_class='multinomial', solver='lbfgs',n_jobs=-1)
#estimator = xgb.XGBClassifier(objective='multi:softprob')
estimator = RandomForestClassifier(n_estimators =20,criterion = 'entropy',random_state = 0)
selector = RFE(estimator, 5, step=1)
selector = selector.fit(df, y)
selector.support_ 


C:\anaconda\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([False,  True,  True,  True, False, False, False, False,  True,
        True, False, False])

In [48]:
selector.ranking_

array([2, 1, 1, 1, 3, 7, 5, 4, 1, 1, 6, 8])

In [49]:
estimator = LogisticRegression(penalty='l2',random_state = 0,class_weight='balanced',multi_class='multinomial', solver='lbfgs',n_jobs=-1)
selector = RFE(estimator, 5, step=1)
selector = selector.fit(df, y)
selector.support_

C:\anaconda\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([False, False, False, False,  True, False,  True,  True, False,
       False,  True,  True])

In [50]:
selector.ranking_

array([5, 3, 7, 4, 1, 2, 1, 1, 6, 8, 1, 1])

In [ ]:
estimator = xgb.XGBClassifier(objective='multi:softprob')
selector = RFE(estimator, 5, step=1)
selector = selector.fit(df, y)
selector.support_

In [ ]:
selector.ranking_

# Applying different Models

In [101]:
#Independent Column
X=df
X.shape

(100000, 92)

In [102]:
#Dependent column
y.shape

(100000, 1)

In [103]:
#splitting the dataset into the training set and test set
#from sklearn import model_selection as ms

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33,random_state=0)

In [104]:
"""from sklearn.preprocessing import StandardScaler
sc_X=StandardScaler()
X_train=sc_X.fit_tranform(X_train)
X_test=sc_X.transform(X_test)"""

'from sklearn.preprocessing import StandardScaler\nsc_X=StandardScaler()\nX_train=sc_X.fit_tranform(X_train)\nX_test=sc_X.transform(X_test)'

# XGBoost Model

In [105]:
X.head()

,X,Y,rot45_X,rot45_Y,radial45,rot30_X,rot30_Y,radial30,rot60_X,rot60_Y,radial60,Intersection,Block,PdDistrict_CENTRAL,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN,Year_2004,Year_2005,Year_2006,Year_2007,Year_2008,Year_2009,Year_2010,Year_2011,Year_2012,Year_2013,Year_2014,Year_2015,Month_Spring,Month_Summer,Month_Winter,Day_02,Day_03,Day_04,Day_05,Day_06,Day_07,Day_08,Day_09,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Hour_EARLY_MORNING,Hour_MORNING,Hour_NIGHT,Minute_low,Address_ALMS,Address_AV,Address_BL,Address_BLOCK,Address_CR,Address_CT,Address_DR,Address_EX,Address_HWY,Address_HY,Address_I-80,Address_LN,Address_MAR,Address_PL,Address_PZ,Address_RD,Address_ST,Address_TER,Address_TR,Address_WAY,Address_WK
0,-122.419331,37.762264,-59.852546,113.248388,128.091860,-28.507545,93.911786,98.143282,-87.134009,124.896273,152.287276,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,-122.411340,37.781271,-59.833459,113.256175,128.089828,-28.487089,93.924250,98.149269,-87.117585,124.898855,152.279998,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,-122.403405,37.775421,-59.831985,113.246430,128.080522,-28.488188,93.915217,98.140944,-87.113638,124.889059,152.269705,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,-122.404715,37.730161,-59.864910,113.215357,128.068437,-28.528038,93.876677,98.115643,-87.137403,124.867564,152.265674,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,-122.405239,37.785265,-59.826321,113.254686,128.085177,-28.480580,93.924659,98.147771,-87.110304,124.895569,152.273137,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [106]:
model = xgb.XGBClassifier(objective='multi:softprob')
model.fit(X_train, y_train)

C:\anaconda\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\anaconda\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [107]:
y_pred=model.predict_proba(X_test)

In [108]:
print (log_loss(y_test,y_pred));

2.429577262798042


Early Stopping in XGBoost

In [ ]:
model = XGBClassifier(objective='multi:softprob')
eval_set = [(X_test, y_test)]
model.fit(X_train, y_train, early_stopping_rounds=10, eval_metric="mlogloss", eval_set=eval_set, verbose=True)

PLot of Important Feature

In [ ]:
# plot feature importance
xgb.plot_importance(model)
